In [1]:
import pandas as pd
import numpy as np
import streamlit as st
import plotly.express as px

In [2]:
# importing datasets
MHP = pd.read_csv('data/MedHome.csv')
MorRate = pd.read_csv('data/MorRate.csv')
MedIncome = pd.read_csv('data/MedIncome.csv')

# converting date to datetime
MHP['DATE'] = pd.to_datetime(MHP['DATE'])
# creates a new column that gives the year for each row
MHP['year'] = MHP['DATE'].dt.year
# Recreates the dataset averating all the given values in the year 
del MHP['DATE']
MHP_Yearly = MHP.groupby('year')['MSPUS'].mean().reset_index()


#doing same for other dataframes
MorRate['date'] = pd.to_datetime(MorRate['date'])
MorRate['year'] = MorRate['date'].dt.year
del MorRate['date']
MorRate_Yearly = MorRate.groupby('year')[' value'].mean().reset_index()

MedIncome['DATE'] = pd.to_datetime(MedIncome['DATE'])
MedIncome['year'] = MedIncome['DATE'].dt.year
del MedIncome['DATE']
MedIncome_Yearly = MedIncome.groupby('year')['MEPAINUSA646N'].mean().reset_index()

# renames the values fields so dataframes can be merged
MorRate_Yearly.rename(columns={' value' : 'Mortgage Rate'}, inplace=1)
MHP_Yearly.rename(columns={'MSPUS': 'Median Home Price'}, inplace=1)
MedIncome_Yearly.rename(columns={'MEPAINUSA646N': 'Median Personal Income'}, inplace=1)

# creating dataframe MYA for 'merged yearly averages'
MYA = pd.merge(pd.merge(MedIncome_Yearly, MHP_Yearly, on='year'), MorRate_Yearly, on='year')

In [3]:
# creating a function to estimate mortgage payments on the median house using avewrage interest rate
def MorPayment(principal, Rate):
    #getting monthly rate in correct format
    MR = Rate/12/100
    return(principal*MR*((1+MR)**360) / ((1+MR)**(360)-1))

In [15]:
# creating the dataframe that has values to be displayed in the app

DF = pd.DataFrame()
DF['Year'] = MYA['year']
DF['Median Personal Income'] = MYA['Median Personal Income']
DF['Median Home Price'] = MYA['Median Home Price']
DF['Home Price/Income Ratio'] = MYA['Median Home Price']/MYA['Median Personal Income'] # column showing how many years of median income it would take to equal the median house value
DF['Monthly Gross Income'] = MYA['Median Personal Income']/12
DF['Average Mortgage Rate'] = MYA['Mortgage Rate']
DF['30YR Mortgage Payment'] = MorPayment(MYA['Median Home Price'], MYA['Mortgage Rate']) # column showing mortgage payments using that years average mortgage rate and median home price (assuming no down payment)
DF['Mortgage/Income Ratio'] = DF['30YR Mortgage Payment']/DF['Monthly Gross Income'] # creates a column showing the percentage of monthly income it would take to cover a mortgage payment



In [16]:
# creating a csv to use in app 

DF.to_csv('data/CleanData.csv', index=0)